In [1]:
from time import sleep
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import os
from selenium.webdriver.support.ui import WebDriverWait
import requests
from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium.common.exceptions import TimeoutException
import datetime
from datetime import datetime, timedelta
from tqdm import tqdm

In [2]:
def scrape_kayak(from_location, to_location, date):

    formatted_date = date.strftime('%Y-%m-%d')
    url = f'https://www.kayak.com/flights/{from_location}-{to_location}/{formatted_date}?sort=bestflight_a'
    driver.get(url)

    
    wait_time = 20  # increase wait time if needed
    wait = WebDriverWait(driver, wait_time)
    
    flight_rows = []

    while True:
        try:
            show_more_buttons = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.ULvh-button.show-more-button')))
            num_buttons = len(show_more_buttons)

            for i in range(num_buttons):
                show_more_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.ULvh-button.show-more-button')))
                driver.execute_script("arguments[0].click();", show_more_button)
                try:
                    wait.until(EC.staleness_of(show_more_button))
                except:
                    pass

            flight_rows = driver.find_elements(By.XPATH, '//div[@class="nrc6-inner"]')

        except TimeoutException:
            break
    
    return flight_rows

In [3]:
def convert_to_df(flight_rows):
    
    list_prices = []
    list_company_names=[]
    list_stops=[]
    list_duration=[]
    list_dates=[]
    
    for WebElement in flight_rows:
        elementHTML = WebElement.get_attribute('outerHTML')
        elementSoup = BeautifulSoup(elementHTML, 'html.parser')
        # price
        temp_price = elementSoup.find("div", {"class": "nrc6-price-section"})
        price = temp_price.find("div", {"class": "f8F1-price-text"})
        list_prices.append(price.text)

        #company names#
        company_names = elementSoup.find("div",{"class":"c_cgF c_cgF-mod-variant-default"}).text
        list_company_names.append(company_names)

        # stops
        temp_stops = elementSoup.find("div", {"class": "vmXl vmXl-mod-variant-default"})
        stops = temp_stops.find("span", {"class": "JWEO-stops-text"})
        list_stops.append(stops.text)

        #durations  
        temp_durations = elementSoup.find("div", {"class": "xdW8"})
        durations = temp_durations.find("div", {"class": "vmXl vmXl-mod-variant-default"}).text
        list_duration.append(durations)
    
    df = pd.DataFrame({
        'Price': list_prices,
        'Company Name': list_company_names,
        'Stops': list_stops,
        'Duration': list_duration,
        })

    df['Destination'] = to_location
    df['From'] = from_location
    df['Date'] = date
    
    return df

In [4]:
driver = webdriver.Chrome()

from_location = 'LAX'
to_location = 'NYC'

date = datetime.strptime('2023-06-01', '%Y-%m-%d')
end_date = datetime.strptime('2023-08-31', '%Y-%m-%d')
total_days = (end_date - date).days + 1

df = pd.DataFrame()
with tqdm(total=total_days) as pbar:

    while date <= end_date:

        df = pd.concat([df, convert_to_df(scrape_kayak(from_location, to_location, date))])
        
        date += timedelta(days=1)
        pbar.update(1)

100%|███████████████████████████████████████████| 92/92 [43:21<00:00, 28.28s/it]


In [5]:
df

,Price,Company Name,Stops,Duration,Destination,From,Date
0,$108,Spirit Airlines,nonstop,5h 29m,NYC,LAX,2023-06-01
1,$189,Alaska Airlines,nonstop,5h 34m,NYC,LAX,2023-06-01
2,$132,Spirit Airlines,1 stop,6h 50m,NYC,LAX,2023-06-01
3,$179,United Airlines,nonstop,5h 11m,NYC,LAX,2023-06-01
4,$179,United Airlines,nonstop,5h 24m,NYC,LAX,2023-06-01
...,...,...,...,...,...,...,...
475,"$2,108",Turkish Airlines,1 stop,30h 55m,NYC,LAX,2023-08-15
476,"$2,108",Turkish Airlines,1 stop,35h 35m,NYC,LAX,2023-08-15
477,"$2,078",Turkish Airlines,1 stop,37h 45m,NYC,LAX,2023-08-15
478,"$2,078",Turkish Airlines,1 stop,42h 25m,NYC,LAX,2023-08-15


In [6]:
df.to_csv('flight_LAX_NYC_data.csv', index=False)